In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
import numpy as np  
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# MNIST veri setini yükleme ve eğitim/test setlerine ayırma
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Verileri CNN modeline uygun formata dönüştürme ve normalizasyon
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


In [ ]:

# CNN modeli oluşturma
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Önceki Dropout katmanı
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Yeni Dropout katmanı
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Bir diğer Dropout katmanı
model.add(Dense(10, activation='softmax'))


In [ ]:
# Modeli derleme
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
# Modeli eğitme
history = model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))

# Modelin başarımını yazdırma
train_acc = history.history['accuracy']
test_acc = history.history['val_accuracy']
# print("Eğitim Doğruluğu:", train_acc)
# print("Test Doğruluğu:", test_acc)

()

In [ ]:

model.save("../models/model_sayilar.h5")

In [ ]:
# Confusion Matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
y_pred = []
y_true = []

for x, y in test_images:
    batch_predictions = model.predict(x)
    y_pred.extend(np.argmax(batch_predictions, axis=1))
    y_true.extend(np.argmax(y, axis=1))
    if len(y_true) >= test_images.n:
        break

conf_matrix = tf.math.confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', xticklabels=list(test_images.class_indices.keys()), yticklabels=list(test_images.class_indices.keys()))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Model ile tahmin yapın
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
# Eğer y_test zaten etiketleri içeriyorsa (one-hot encoded değilse)
y_true = test_labels

In [ ]:

# Metrikleri hesaplayın
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Sonuçları yazdırın
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Detaylı sınıflandırma raporu
print(classification_report(y_true, y_pred_classes))

In [ ]:

# Kayıp ve doğruluk grafiklerini çizmek için
def plot_history(history):
    # Kayıp
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Eğitim Kaybı')
    plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
    plt.xlabel('Epoch')
    plt.ylabel('Kayıp')
    plt.legend()
    plt.title('Eğitim ve Doğrulama Kaybı')

    # Doğruluk
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
    plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
    plt.xlabel('Epoch')
    plt.ylabel('Doğruluk')
    plt.legend()
    plt.title('Eğitim ve Doğrulama Doğruluğu')
    
    plt.show()

# history nesnesini kullanarak grafik çizme
plot_history(history)
